### Introduction 

### Setup

In [ ]:
import pymongo
import pandas as pd
from pandas.io.json import json_normalize 
from pymongo import MongoClient
import numpy as np 

In [ ]:
 #point the client at mongo URI 
client = MongoClient('mongodb://localhost:27017')
#select database
db = client['tipster']

Two mongodb collections - one for results meeting-horses-place , one for form - horse

In [ ]:
form_collection = db.form
data = form_collection.find({},{"_id":0}) 
form_df = pd.DataFrame(json_normalize(data, 'form',['horse_name']))

In [ ]:
# flatten data and put into dataframe 
meetings_collection = db.meetings
data=meetings_collection.find({},{"_id":0})
meeting_runners_df = pd.DataFrame(json_normalize(data,'runners',['meeting_url','meeting_date_time']))
mdf=meeting_runners_df

##### generate dependent variable series with indexed by horse_name-meeting_url so they can be joined together painlessly

#####  has winning form
Create series indexed by horse_name-meeting_url for whether the horse has won prior to the meeting data: - 
Find first win date of all winning horses - date of earlist winning meeting for each horse  
Bring back all horses, winners or not using left outer join  
###### some name-mangling by merge() -  
meeting_date_time_x = current meeting date time  
meeting_date_time_y = earliest win  
###### ... finally
generate dummy variable current meeting date time strictly greater than earliest win - every date > NaT

In [ ]:
earliest_win_date = mdf[mdf.place==1].groupby(mdf.horse_name).meeting_date_time.min().reset_index()
tmp = pd.merge(mdf[['horse_name','meeting_url','meeting_date_time']],\
               earliest_win_date, on='horse_name',how='left').set_index(['horse_name','meeting_url'])
tmp['has_winning_form']  = np.where(tmp.meeting_date_time_x > tmp.meeting_date_time_y,1,0)
has_winning_form = tmp['has_winning_form']

#### Form against competitors
Generate pairs of horse in every race- cartesian product of horses for each meeting -  
also need meeting date time so we can just count results previous to meeting when we do merge  

'Winner' is horse that is better placed then other horse - 'loser'

if date of win is prior to meeting date update beaten other horse in race in 'x' to true

In [ ]:
mdfl = meeting_runners_df[['meeting_url','horse_name','place','meeting_date_time']]
mdfr = mdfl
horse_pairs = mdfl.merge(mdfr, on='meeting_url')

In [ ]:
horse_pairs = horse_pairs[horse_pairs.horse_name_x != horse_pairs.horse_name_y]
horse_pairs['winner'] = np.where(horse_pairs.place_x < horse_pairs.place_y,
                                 horse_pairs.horse_name_x, horse_pairs.horse_name_y )
horse_pairs['loser'] = np.where(horse_pairs.place_x < horse_pairs.place_y,
                                 horse_pairs.horse_name_y, horse_pairs.horse_name_x )

#### collateral form   
##### if A beat B and B beat C in another race , then A has collateral form against C
if A and C are competitors in the same race 
if A is a winner and B is a loser in the most race they ran togther, and B won against C in the most recent race they ran together before that, then A has collateral form against C. 



if B and C are competitors in the same race 
if B lost against A in the most previous race they ran together and C lost against A in the most previous race they ran together, and B was better placed than C then A has collateral form against C.

##### if A beat B in one meeting and A beat C in another meeting and B had a better place than C against A, then B has indirect form against C 



In [ ]:
#join to get horse A,B and C  
collateral_form = horse_pairs.merge(horse_pairs, left_on="loser", right_on="winner")
#don't include three horses in same race
collateral_form = collateral_form[collateral_form['meeting_url_x']!=collateral_form['meeting_url_y']]
#form is retrospective! 
collateral_form = collateral_form[collateral_form['meeting_date_time_x_x']>=collateral_form['meeting_date_time_y_y']]

In [ ]:
# sort for merge_asof
cf1 = collateral_form[['winner_x','loser_y','meeting_date_time_y_y','meeting_url_x']].sort_values(by='meeting_date_time_y_y')
cm1 = horse_pairs[['horse_name_x','horse_name_y','meeting_date_time_y']].sort_values(by='meeting_date_time_y')

#pairs of horses in same race - match first horse with horse A - winner and second horse with horse C 
#using most recent race where horse C was beaten 
collateral_form_pairs = pd.merge_asof(cm1,cf1,allow_exact_matches=False, \
                    left_by=['horse_name_x','horse_name_y'], right_by=['winner_x','loser_y'], \
                    left_on='meeting_date_time_y', right_on='meeting_date_time_y_y')

#outer join so remove nulls - can't find matching horse C 
collateral_form_pairs = collateral_form_pairs[collateral_form_pairs['winner_x'].notnull()] 

In [ ]:
mrd = meeting_runners_df
has_cf = pd.merge(collateral_form_pairs, mrd, left_on=['winner_x','meeting_url_x'],\
                  right_on=['horse_name','meeting_url'],how='right')

horse may have beaten a horse that beat more than one  horse in the same race

In [ ]:
has_cf =  has_cf[['horse_name', 'meeting_url','meeting_date_time_y_y']].groupby(['horse_name', 'meeting_url']).meeting_date_time_y_y.agg("max").reset_index()
has_cf['franking_form'] = np.where(has_cf.meeting_date_time_y_y.isnull(),0,1)
has_cf



### Data|

### Features


which horses in a meeting are winners as of meeting date?  

has a horse in a meeting beaten another horse in the same meeting?

for a given pair of horses which one did better against a third horse?  
    how similar were the conditions? 

time since last race (fitness and value of form)

does the horse need to improve - is it a champion

will form improve?   
margin of victory - ( bigger implies form won't change )  (low fitness + ease => improving form) 

commentary:  
   activity during race  
   finish  
   speed 

is progressively better

above or below average time for course?

raised or lowered in class?

proven at distance 

proven for going

weight

draw - look up advantage of draw in course

type of race - handicap/flat/jumps

### Training


In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
# from sklearn.cross_validation import train_test_split 



### Predict